# **Fraud Prediction**

## **Problem Statement**

💼 **Fraud** merupakan suatu tindakan penipuan pada transaksi keuangan. Transaksi ini bertujuan untuk memperoleh keuntungan dari suatu entitas lain dengan cara ilegal sehingga dapat menyebabkan kerugian. 

Transaksi fraud sering terjadi di industri perbankan. Beberapa contoh transaksi fraud pada industri perbankan antara lain adalah *pishing, skimming*, penipuan kartu kredit, penipuan pinjaman, dll.

Efek negatif dari transaksi fraud adalah adanya kerugian finansial di kedua belah pihak baik customer maupun industri perbankan. Oleh karena itu diperlukan tindakan pencegahan dengan deteksi lebih dini terhadap potensi transaksi fraud. 

Transaksi fraud dapat dideteksi lebih awal dengan cara membangun model *machine learning* dengan metode klasifikasi. Tugas kita pada project kali ini adalah membuat model klasifikasi untuk mendeteksi transaksi fraud agar dapat dilakukan action plan berikutnya

## **Rubrics Penilaian**

Untuk menyelesaikan project ini, silahkan mengacu pada rubrics di bawah ini :

***Data Preparation***

(1 poin) Bagaimana melakukan persiapan data sebelum dilakukan pemodelan.

- Metode apa saja yang yang dilakukan dalam proses persiapan data?
- Apakah terdapat data yang *missing* atau *duplicate*, bagaimana cara mengatasi data tersebut?

(1 poin) Bagaimana cara untuk melakukan *feature engineering* ataupun pemilihan variabel dari data yang tersedia.

- Apakah terdapat variable yang dihilangkan? Dan kenapa?
- Apakah terdapat variable yang ditambahkan? Dan kenapa?

***Exploratory Data Analysis***

(2 poin) *Exploratory Data Analysis*

- Berikan penjelasan informatif dari visualisasi dan/atau segala jenis hasil eksplorasi Anda.
- Bagaimana distribusi data pada setiap variabel?
- Apakah terdapat informasi menarik antara variable predictor dengan variable target?

***Data Pre-processing and Model Fitting***

(1 poin) *Data Pre-processing*

- Apakah data categoricalnya akan ditreatement menjadi ordinal atau nominal encoding?

(1 poin) *Cross Validation*

- Bagaimana pembagian proporsi Data Train & Test?
- Apakah proporsi target variable pada data train dibilang cukup stabil? Apakah kita masih harus melakukan *Balancing Data*?

(4 poin) Membuat model

- Buatlah 2 model yang dapat menjawab pertanyaan bisnis di atas.
- Metode evaluasi apa yang akan digunakan?
- Apakah modelnya overfit, underfit, just right?
- Variable apa yang dirasa cukup penting dalam pembuatan model? Sertakan alasannya.

***Prediction Performance***

- (1 poin) **Precision** pada dataset **train** mencapai> 80%
- (1 poin) **Recall** pada dataset **train** mencapai > 80%
- (2 poin) **Precision** pada dataset **test** mencapai > 80%.
- (2 poin) **Recall** pada dataset **test** mencapai > 80%.

***Conclusion***

(2 poin) Tuliskan kesimpulan dari project yang anda kerjakan. 

- Apakah model sudah dapat melakukan prediksi dengan baik?
- Apakah model sudah dapat menjawab pertanyaan bisnis yang ada?
- Action plan apa yang dapat dilakukan untuk tindakan preventif transaksi fraud?

**Total Poin Capstone Project : 18**

## **Workflow Pengerjaan**

### ***Data Preparation***

**Import Library**

In [181]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.metrics import recall_score, precision_score, accuracy_score

**Read Data**

In [ ]:
# Read Data dari VSC
fraud_data = pd.read_csv('data/fraud_data.csv')
fraud_data.head()

(1 poin) Bagaimana melakukan persiapan data sebelum dilakukan pemodelan.

- Metode apa saja yang yang dilakukan dalam proses persiapan data?
>Merapikan Kolom dan Mengubah tipe data, 
- Apakah terdapat data yang *missing* atau *duplicate*, bagaimana cara mengatasi data tersebut?
>tidak ada missing value atau duplicate. cara mengatasinya adalah dengan melakukan imputasi dan di drop. sedangkan duplicate dapat di drop

In [ ]:
# pengecekan tipe data
fraud_data.info()

In [184]:
## menyalin data merubah tipe data dan melakukan set index
fraud_data_copy = fraud_data.copy()

fraud_data_copy['type'] = fraud_data_copy['type'].astype("category")

fraud_data_copy = fraud_data_copy.set_index('nameOrig')


In [ ]:
# pengecekan tipe data kembali
fraud_data_copy.info()

In [ ]:
##  melakukan pengecekan data null dan duplikat
print(fraud_data_copy.isnull().sum())
print(fraud_data_copy.duplicated().sum())


(1 poin) Bagaimana cara untuk melakukan *feature engineering* ataupun pemilihan variabel dari data yang tersedia.

- Apakah terdapat variable yang dihilangkan? Dan kenapa?
>ya. variabel nameDest dihilangkan karena tidak dapat digunakan untuk metode perhitungan
- Apakah terdapat variable yang ditambahkan? Dan kenapa?
>penambahan variabel type menggunakan pd.get_dummies karena variabel type bersifat nominal sehingga dapat harus dilakukan encoding


In [187]:
# penghapusan kolom yang tidak diperlukan da
fraud_data_copy = fraud_data_copy.drop(columns='nameDest')

### ***Exploratory Data Analysis***

(2 poin) *Exploratory Data Analysis*

- Berikan penjelasan informatif dari visualisasi dan/atau segala jenis hasil eksplorasi Anda.
> 1. oldbalancedest di dominasi data  0. ini menunjukan kebanyakan transaksi tertuju pada rekening kosong
> 2. terdapat amount 0. yang berarti terdapat transaksi yang tidak valid
> 3. data fraud hanya dilakukan menggunakan type CASH_OUT dan TRANSFER
> 4. korelasi yang paling tinggi terhadap fraud adalah data amount

- Bagaimana distribusi data pada setiap variabel?
> 1. pada data ini terdapat beberapa data yang memiliki kesamaan dalam distribusi datanya terlebih pada kolom oldbalanceDest dan newBalanceDest
> 2. data berkumpul di luar kotak whiskers. ini menandakan banyak data yang outlier

- Apakah terdapat informasi menarik antara variable predictor dengan variable target?
> untuk menjawab hal ini, saya menggunakan corr() untuk menghitung koefisiensi korelasi antar variaber predictor dan variabel target
> - dari pengecekan korelasi mendapatkan hasil bahwa variabel AMOUNT dan TYPE_CASHOUT lah yang memiliki korelasi yang paling kuat dari data target


In [ ]:
# pengecekan menggunakan describe
fraud_data_copy[fraud_data_copy['isFraud']==1].describe()

In [ ]:
# pengecekan sebaran data
pd.crosstab(index= fraud_data_copy['type'], columns= fraud_data_copy['isFraud'])

In [ ]:
# cek korelasi antar variabel predictor dengan target
fraud_enc_real.corr()

### ***Data Pre-processing and Model Fitting***

(1 poin) *Data Pre-processing*

- Apakah data categoricalnya akan ditreatement menjadi ordinal atau nominal encoding?
>iya. melakukan encoding pada data variabel "type" menjadi nominal

In [191]:
## melakukan encoding pada variabel type menjadi nominal
fraud_enc = pd.get_dummies(data = fraud_data_copy, 
                             columns = ['type'],
                             drop_first = True,
                             dtype =float)
fraud_enc_real = fraud_enc.copy()

(1 poin) *Cross Validation*

- Bagaimana pembagian proporsi Data Train & Test?
> pembagian menggunakan train_test_split dengan pembagian 70% data training dan 30% data test
- Apakah proporsi target variable pada data train dibilang cukup stabil? Apakah kita masih harus melakukan *Balancing Data*?
> porposi tidak seimbang antara data fraud dengan tidak fraud yaitu 2793 data fraud, dan 4207 data tidak fraud. oleh karena itu dibutuhkan metode downsampling, oversampling ataupun smote untuk melakukan balancing data. dan didalam model ini saya akan menggunakan smote untuk melakukan balancing data

In [192]:
# melakukan pembagian antara target dan prediktor
import statsmodels.api as sm

target = fraud_enc['isFraud']

prediktor = fraud_enc.drop(columns= 'isFraud')

In [193]:
# melakukan pembagian data train dan test 
X_train, X_test, y_train, y_test = train_test_split(prediktor, 
                                                    target, 
                                                    test_size = 0.3, 
                                                    random_state = 10)

In [ ]:
# cek proporsi data
pd.crosstab(index = y_train, columns = 1)

In [172]:
# melakukan balancing data menggunakan smote sampling
from imblearn.over_sampling import SMOTENC

smote_nc = SMOTENC(categorical_features=[3, 4, 5, 6], random_state=0)
X_train_smote, y_train_smote = smote_nc.fit_resample(X_train, y_train)

(4 poin) Membuat model

- Buatlah 2 model yang dapat menjawab pertanyaan bisnis di atas.
> saya menggunakan 2 model yaitu decision tree dan random forest
- Metode evaluasi apa yang akan digunakan?
> menggunakan precision dan recall. selain itu saya juga menggunakan oob_score untuk menhitung akurasi dari random forest 
- Apakah modelnya overfit, underfit, just right?
> menggunakan decision tree, hasilnya menjadi overfit karena itu juga adalah kelemahan dari model tersebut. oleh karena itu saya melakukan pruning untuk menghadapi hal tersebut.
> sedangkan menggunakan random forest hasilnya sudah optimal
- Variable apa yang dirasa cukup penting dalam pembuatan model? Sertakan alasannya.
>untuk menjawab hal ini, saya menggunakan metode feature_importan dari 2 model yang dibuat untuk mengetahui variabel yang memiliki pengaruh yang signifikan
> - feature_importances_ Decision Tree = menjelaskan bahwa variabel OLDBALANCE, AMOUNT, TYPE_PAYMENT lah yang paling berpengaruh dari model tersebut
> - feature_importances_ Random Forest = menjelaskan bahwa variabel AMOUNT, TYPE_TRANSFER, NEW_BALANCE lah yang paling berpengaruh dari model tersebut
> 
>kesimpulan yang bisa diambil adalah bahwa variabel AMOUNT lah yang paling penting dalam pembuatan model ini

In [ ]:
# membuat 2 model yaitu model Decision Tree dan Random Forest

# - Decision Tree
from sklearn.tree import DecisionTreeClassifier
DT_smote = DecisionTreeClassifier(criterion='entropy', random_state=10)
DT_smote = DT_smote.fit(X_train_smote, y_train_smote)

# - Decision Tree + pruning
DT_tuning = DecisionTreeClassifier(
    max_depth=5,
    min_samples_split=10,
    min_samples_leaf=5,
    max_leaf_nodes=None,
    random_state=123)

DT_tuning.fit(X_train_smote, y_train_smote)


In [ ]:
# - Random Forest
from sklearn.ensemble import RandomForestClassifier
Model_rf = RandomForestClassifier(random_state=500, 
                                   oob_score = True, 
                                   n_estimators= 500, #jumlah dt
                                   max_depth=50,
                                   min_samples_split=10,
                                   min_samples_leaf=5,
                                   max_leaf_nodes=None)

Model_rf.fit(X_train, y_train)

In [ ]:
# cek variabel yang penting dari decesion tree
import plotly.express as px
var_importance = \
pd.Series(
    DT_smote.feature_importances_, 
    index=X_train_smote.columns 
    ).sort_values(ascending=False).tail(10) 

px.bar(var_importance)
# oldbalance, amount, type_payment

In [ ]:

# cek variabel yang penting dari random forest

Model_rf.feature_importances_
import plotly.express as px
var_importance = \
pd.Series(
    Model_rf.feature_importances_, 
    index=X_train.columns 
    ).sort_values(ascending=False).tail(10) 

px.bar(var_importance)
# amount, type_transfer, new_balance, old_balance

In [ ]:
# pengecekan oob_score untuk random forest
Model_rf.oob_score_

### ***Prediction Performance***

- (1 poin) **Precision** pada dataset **train** mencapai> 80%

- (1 poin) **Recall** pada dataset **train** mencapai > 80%

- (2 poin) **Precision** pada dataset **test** mencapai > 80%.

- (2 poin) **Recall** pada dataset **test** mencapai > 80%.

In [ ]:
# Prediksi DecisionTree terhadap data test
y_pred_dt_test = DT_smote.predict(X_test)

print(f'dt test Recall score: {recall_score(y_test, y_pred_dt_test)}')
print(f'dt test Precision score: {precision_score(y_test, y_pred_dt_test)}')

# Prediksi DecisionTree terhadap data train
ytrain_pred_dt_train = DT_smote.predict(X_train_smote)

print(f'dt train Recall score: {recall_score(y_train_smote, ytrain_pred_dt_train)}')
print(f'dt train Precision score: {precision_score(y_train_smote, ytrain_pred_dt_train)}')

In [ ]:
# Prediksi DecisionTree yang sudah di pruning terhadap data test
y_pred_tuning_test = DT_tuning.predict(X_test)

print(f'dt test Recall score: {recall_score(y_test, y_pred_tuning_test)}')
print(f'dt test Precision score: {precision_score(y_test, y_pred_tuning_test)}')

# Prediksi DecisionTree terhadap data train
ytrain_pred_dt_train = DT_tuning.predict(X_train_smote)

print(f'dt train Recall score: {recall_score(y_train_smote, ytrain_pred_dt_train)}')
print(f'dt train Precision score: {precision_score(y_train_smote, ytrain_pred_dt_train)}')

In [ ]:
# prediksi Randomforest terhadap data test

pred_test_rf = Model_rf.predict(X_test)

print(f'RF test Recall score: {recall_score(y_test, pred_test_rf)}')
print(f'RF test Precision score: {precision_score(y_test, pred_test_rf)}')


# Prediksi Randomforest terhadap data train

pred_train_rf = Model_rf.predict(X_train)

print(f'RF train Recall score: {recall_score(y_train, pred_train_rf)}')
print(f'RF train Precision score: {precision_score(y_train, pred_train_rf)}')


### ***Conclusion***

(2 poin) Tuliskan kesimpulan dari project yang anda kerjakan. 

- Apakah model sudah dapat melakukan prediksi dengan baik? 
> iya. model sudah dapat melakukan prediksi degan baik. menggunakan decision tree maupun random forest. bisa dilihat dari nilai recall dan precision serta oob score yang tinggi diatas 80%. namun untuk kasus decision tree, model melakukan prediksi yang overfit sehingga perlu di pruning
- Apakah model sudah dapat menjawab pertanyaan bisnis yang ada?
>saya rasa iya, karena model sudah dapat melakukan prediksi yang tinggi terhadap transaksi fraud
- Action plan apa yang dapat dilakukan untuk tindakan preventif transaksi fraud?
> menurut saya. dari hasil model diatas telah melakukan prediksi yang baik. sehingga model diatas sudah dapat diimplementasikan kedalam production seperti sebuah sistem deteksi transaksi fraud otomatis atau sistem monitoring real-time. namun ada baiknya agar selalu melakukan evaluasi Secara Berkala dari model mengenai tingkat performanya